In [1]:

# mount drive https://datascience.stackexchange.com/questions/29480/uploading-images-folder-from-my-system-into-google-colab
# login with your google account and type authorization code to mount on your google drive.
import os
from google.colab import drive
drive.mount('/gdrive')


Mounted at /gdrive


In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from pandas.core.common import flatten
import copy
import numpy as np
import random

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

import glob
from tqdm import tqdm

In [5]:
torch.tensor([[1.,0,0]]).size()

torch.Size([1, 3])

In [18]:
import numpy
import pandas as pd
import os
import torch
root2= '/gdrive/MyDrive/project/car-damage-dataset/data2a'
training = root2+'/training'
# we then split the training in train, valid so we store everything in the training folder
front = '/00-front/'
rear = '/01-rear/'
side = '/02-side/'

device = ("cuda" if torch.cuda.is_available() else "cpu")

train_df = pd.DataFrame(columns=["img_name","label"])
train_df_2 = pd.DataFrame(columns=["img_name","label"])
train_df_3 = pd.DataFrame(columns=["img_name","label"])



max_idx=0
max_idx_2=0
train_df["img_name"] = os.listdir(training+front)
for idx, i in enumerate(os.listdir(training+front)):
    train_df['label'][idx] = '1 0 0'
    # if idx>max_idx:
    #   max_idx = idx

# print(train_df)

train_df_2["img_name"] = os.listdir(training+rear)
for idx, i in enumerate(os.listdir(training+rear)):
    train_df_2['label'][idx] = '0 1 0'


train_df_3["img_name"] = os.listdir(training+side)
for idx, i in enumerate(os.listdir(training+side)):
    train_df_3['label'][idx] = '0 0 1'


# print(train_df_2)
frames = [train_df,train_df_2, train_df_3]
result_train_df = pd.concat(frames)
result_train_df=result_train_df.sample(frac=1).reset_index(drop=True)
print(result_train_df)
result_train_df.to_csv(r'/gdrive/MyDrive/project/'+'damage_locations_4.csv', index = False, header=True)


       img_name  label
0     0009.JPEG  1 0 0
1     0114.jpeg  0 1 0
2     0211.jpeg  0 1 0
3     0159.JPEG  1 0 0
4     0189.JPEG  1 0 0
...         ...    ...
1145  0332.JPEG  1 0 0
1146  0113.JPEG  0 0 1
1147  0299.JPEG  1 0 0
1148  0114.JPEG  0 0 1
1149  0237.JPEG  0 1 0

[1150 rows x 2 columns]


In [7]:
print(result_train_df)

           img_name                                   label
0         0041.JPEG  [[tensor(0.), tensor(1.), tensor(0.)]]
1         0135.JPEG  [[tensor(0.), tensor(0.), tensor(1.)]]
2         0088.JPEG  [[tensor(0.), tensor(1.), tensor(0.)]]
3         0339.JPEG  [[tensor(1.), tensor(0.), tensor(0.)]]
4         0087.JPEG  [[tensor(0.), tensor(0.), tensor(1.)]]
...             ...                                     ...
1145      0114.JPEG  [[tensor(1.), tensor(0.), tensor(0.)]]
1146      0407.JPEG  [[tensor(1.), tensor(0.), tensor(0.)]]
1147  0021 (1).JPEG  [[tensor(0.), tensor(0.), tensor(1.)]]
1148  0019 (1).JPEG  [[tensor(1.), tensor(0.), tensor(0.)]]
1149      0105.jpeg  [[tensor(0.), tensor(1.), tensor(0.)]]

[1150 rows x 2 columns]


In [13]:
from torch.utils.data import Dataset
import pandas as pd
import os
from PIL import Image
import torch

class Damage_location(Dataset):
    def __init__(self, root_dir_front,root_dir_side, root_dir_rear, annotation_file, transform=None):
        self.root_dir1 = root_dir_front
        self.root_dir2 = root_dir_rear
        self.root_dir3 = root_dir_side
        self.annotations = pd.read_csv('/gdrive/MyDrive/project/'+annotation_file)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        value = self.annotations.iloc[index, 1]
        if value == torch.tensor([1.,0,0]):
          root_dir = self.root_dir1
        elif value == torch.tensor([0,0,1.]):
          root_dir = self.root_dir2
        else:
          root_dir = self.root_dir3
        img = Image.open(os.path.join(root_dir, img_id)).convert("RGB")
        y_label = torch.tensor(float(self.annotations.iloc[index, 1]))

        if self.transform is not None:
            img = self.transform(img)

        return (img, y_label)

In [9]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

from tqdm import tqdm
device = ("cuda" if torch.cuda.is_available() else "cpu")


In [10]:
transform = transforms.Compose(
        [
            transforms.Resize((356, 356)),
            transforms.RandomCrop((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

In [11]:
num_epochs = 40
learning_rate = 0.00001
train_CNN = False
batch_size = 32
shuffle = True
pin_memory = True
num_workers = 1
%store learning_rate

Stored 'learning_rate' (float)


In [14]:
dataset = Damage_location(training+front,training+side,training+rear,"damage_locations_2.csv",transform=transform)

train_set, validation_set = torch.utils.data.random_split(dataset,[1000,150])
train_loader = DataLoader(dataset=train_set, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers,pin_memory=pin_memory)
validation_loader = DataLoader(dataset=validation_set, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers, pin_memory=pin_memory)
from google.colab import files
# files.download('dataset')

# %store dataset

In [ ]:
def get_variable(): 
  return dataset,train_set,validation_set,train_loader,validation_loader

In [ ]:
%store dataset


Proper storage of interactively declared classes (or instances
of those classes) is not possible! Only instances
of classes in real modules on file system can be %store'd.



In [ ]:
import pickle

a = [dataset,train_set,train_loader,validation_loader]

# Choose a file name
file_name = "sharedfile"

# Open the file for writing
with open(file_name,'wb') as my_file_obj:
    pickle.dump(a,my_file_obj)   

# The file you have just saved can be opened in a different session
# (or iPython notebook) and the contents will be preserved.

# # Now select the (same) file to open (e.g. in another notebook)
# file_name = "sharedfile"
# # Open the file for reading
# file_object = open(file_Name,'r')  
# # load the object from the file into var b
# b = pickle.load(file_object)  

# print(b)
# >>> ['test value','test value 2','test value 3']